In [2]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine
import pandas as pd

In [4]:
wine = load_wine()
X = wine.data
y = wine.target
feature_names = wine.feature_names

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42, stratify=y)

print("Train:", X_train.shape, "Test:", X_test.shape)